In [1]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from shapely.ops import unary_union
import pandas as pd
from typing import List, Tuple, Optional
from dataclasses import dataclass

@dataclass
class SimulationResults:
    n_found: int
    path_length: float
    found_plants: List[Point]
    detection_rate: float

class PlotSimulation:
    def __init__(self, width: float = 10.0, height: float = 10.0, n_plants: int = 20,
                 distribution: str = 'random', distribution_params: dict = None,
                 detection_params: dict = None):
        """Initialize plot simulation with customizable parameters"""
        self.width = width
        self.height = height
        self.n_plants = n_plants
        self.distribution = distribution
        self.distribution_params = distribution_params or {}
        
        if detection_params is None:
            self.detection_params = {'fixed': 0.2}
        elif isinstance(detection_params, (int, float)):
            self.detection_params = {'fixed': float(detection_params)}
        else:
            self.detection_params = detection_params
            
        self.plants = self._generate_plants()
        self.plant_buffers = self._create_plant_buffers()

    def _generate_plants(self) -> List[Point]:
        """Generate plant locations based on specified distribution."""
        distribution_methods = {
            'random': self._generate_random_plants,
            'clustered': self._generate_clustered_plants,
            'regular': self._generate_regular_plants,
            'gradient': self._generate_gradient_plants
        }
        
        if self.distribution not in distribution_methods:
            raise ValueError(f"Unknown distribution type: {self.distribution}")
            
        return distribution_methods[self.distribution]()
    
    def _generate_random_plants(self) -> List[Point]:
        """Generate completely random plant locations."""
        x_coords = np.random.uniform(0, self.width, self.n_plants)
        y_coords = np.random.uniform(0, self.height, self.n_plants)
        return [Point(x, y) for x, y in zip(x_coords, y_coords)]
    
    def _generate_clustered_plants(self) -> List[Point]:
        """Generate clustered plant distribution using Gaussian clusters."""
        n_clusters = self.distribution_params.get('n_clusters', 3)
        cluster_std = self.distribution_params.get('cluster_std', 1.0)
        
        cluster_centers_x = np.random.uniform(cluster_std, self.width - cluster_std, n_clusters)
        cluster_centers_y = np.random.uniform(cluster_std, self.height - cluster_std, n_clusters)
        
        points = []
        plants_per_cluster = self.n_plants // n_clusters
        remaining_plants = self.n_plants % n_clusters
        
        for i in range(n_clusters):
            n_plants_this_cluster = plants_per_cluster + (1 if i < remaining_plants else 0)
            x_coords = np.random.normal(cluster_centers_x[i], cluster_std, n_plants_this_cluster)
            y_coords = np.random.normal(cluster_centers_y[i], cluster_std, n_plants_this_cluster)
            x_coords = np.clip(x_coords, 0, self.width)
            y_coords = np.clip(y_coords, 0, self.height)
            points.extend([Point(x, y) for x, y in zip(x_coords, y_coords)])
            
        return points
    
    def _generate_regular_plants(self) -> List[Point]:
        """Generate regularly spaced plants with optional jitter."""
        jitter = self.distribution_params.get('jitter', 0.2)
        n_rows = int(np.sqrt(self.n_plants * self.height / self.width))
        n_cols = int(np.sqrt(self.n_plants * self.width / self.height))
        row_spacing = self.height / (n_rows + 1)
        col_spacing = self.width / (n_cols + 1)
        
        points = []
        for i in range(1, n_rows + 1):
            for j in range(1, n_cols + 1):
                if len(points) < self.n_plants:
                    x = j * col_spacing + np.random.uniform(-jitter, jitter)
                    y = i * row_spacing + np.random.uniform(-jitter, jitter)
                    x = np.clip(x, 0, self.width)
                    y = np.clip(y, 0, self.height)
                    points.append(Point(x, y))
        return points
    
    def _generate_gradient_plants(self) -> List[Point]:
        """Generate plants with density following a gradient."""
        direction = self.distribution_params.get('direction', 'x')
        steepness = self.distribution_params.get('steepness', 1.0)
        
        points = []
        while len(points) < self.n_plants:
            x = np.random.uniform(0, self.width)
            y = np.random.uniform(0, self.height)
            
            if direction == 'x':
                prob = np.exp(-steepness * x / self.width)
            elif direction == 'y':
                prob = np.exp(-steepness * y / self.height)
            else:
                prob = np.exp(-steepness * (x + y) / (self.width + self.height))
                
            if np.random.random() < prob:
                points.append(Point(x, y))
        return points
    
    def _create_plant_buffers(self):
        """Create detection buffers around plants with variable radii."""
        if 'fixed' in self.detection_params:
            radius = self.detection_params['fixed']
            return [plant.buffer(radius) for plant in self.plants]
            
        dist_type = self.detection_params.get('distribution', 'uniform')
        params = self.detection_params.get('params', {})
        
        if dist_type == 'uniform':
            min_r = params.get('min', 0.1)
            max_r = params.get('max', 0.3)
            radii = np.random.uniform(min_r, max_r, self.n_plants)
        elif dist_type == 'normal':
            mean = params.get('mean', 0.2)
            std = params.get('std', 0.05)
            radii = np.clip(np.random.normal(mean, std, self.n_plants), 0.01, None)
        elif dist_type == 'lognormal':
            mean = params.get('mean', -2)
            std = params.get('std', 0.5)
            radii = np.exp(np.random.normal(mean, std, self.n_plants))
        else:
            raise ValueError(f"Unknown detection radius distribution: {dist_type}")
            
        return [plant.buffer(radius) for plant, radius in zip(self.plants, radii)]

    def parallel_path(self, spacing: float = 2.0) -> LineString:
        """Generate parallel line search path."""
        n_traverses = max(2, int(np.ceil(self.height / spacing)) + 1)
        points = []
        
        for i in range(n_traverses):
            y = i * spacing
            if y <= self.height:
                points.extend([(0, y), (self.width, y)])
        
        return LineString(points)

    def zigzag_path(self, spacing: float = 2.0, angle_degrees: float = 0) -> LineString:
        """Generate zigzag search path with rotation."""
        n_traverses = max(2, int(np.ceil(self.height / spacing)) + 1)
        points = []
        
        for i in range(n_traverses):
            y = i * spacing
            if y <= self.height:
                if i % 2 == 0:
                    points.extend([(0, y), (self.width, y)])
                else:
                    points.extend([(self.width, y), (0, y)])
        
        if angle_degrees == 0:
            return LineString(points)
        
        angle = np.radians(angle_degrees)
        center_x, center_y = self.width/2, self.height/2
        rotated_points = []
        
        for x, y in points:
            tx = x - center_x
            ty = y - center_y
            rx = tx * np.cos(angle) - ty * np.sin(angle)
            ry = tx * np.sin(angle) + ty * np.cos(angle)
            rotated_points.append((rx + center_x, ry + center_y))
        
        return LineString(rotated_points)

    def evaluate_path(self, path: LineString) -> SimulationResults:
        """Evaluate a search path."""
        found_plants = []
        for plant, buffer in zip(self.plants, self.plant_buffers):
            if path.intersects(buffer):
                found_plants.append(plant)
        
        return SimulationResults(
            n_found=len(found_plants),
            path_length=path.length,
            found_plants=found_plants,
            detection_rate=len(found_plants) / self.n_plants
        )

    def plot_simulation(self, path: Optional[LineString] = None, 
                       found_plants: Optional[List[Point]] = None,
                       title: Optional[str] = None):
        """Plot the simulation state."""
        fig, ax = plt.subplots(figsize=(10, 10))
        
        # Plot plants and buffers
        for plant, buffer in zip(self.plants, self.plant_buffers):
            ax.plot(plant.x, plant.y, 'g.', markersize=8)
            x, y = buffer.exterior.xy
            ax.plot(x, y, 'g--', alpha=0.3)
        
        # Highlight found plants
        if found_plants:
            for plant in found_plants:
                ax.plot(plant.x, plant.y, 'r*', markersize=12)
        
        # Plot search path
        if path:
            x, y = path.xy
            ax.plot(x, y, 'b-', label='Search Path', linewidth=2)
        
        ax.set_xlim(-0.5, self.width + 0.5)
        ax.set_ylim(-0.5, self.height + 0.5)
        ax.grid(True)
        ax.set_aspect('equal')
        
        if title:
            ax.set_title(title)
        else:
            ax.set_title(f'Plot Simulation ({self.width}m x {self.height}m)')
        
        ax.set_xlabel('Width (m)')
        ax.set_ylabel('Height (m)')
        
        if path:
            plt.legend()
        
        plt.show()

In [7]:
class SearchOptimization:
    def __init__(self, plot_params: dict):
        """Initialize search pattern optimization."""
        self.plot_params = plot_params
        self.results = []

    def run_parameter_sweep(self, 
                          n_trials: int = 5,
                          spacings: List[float] = None,
                          angles: List[float] = None):
        """Run systematic tests of different search parameters."""
        if spacings is None:
            spacings = [0.5, 1.0, 1.5, 2.0, 2.5]
        if angles is None:
            angles = [0, 15, 30, 45]
        
        self.results = []
        
        for trial in range(n_trials):
            sim = PlotSimulation(**self.plot_params)
            n_total_plants = sim.n_plants
            
            # Test parallel paths
            for spacing in spacings:
                path = sim.parallel_path(spacing=spacing)
                results = sim.evaluate_path(path)
                
                self.results.append({
                    'trial': trial,
                    'pattern': 'parallel',
                    'spacing': spacing,
                    'angle': None,
                    'n_found': results.n_found,
                    'n_total': n_total_plants,
                    'detection_rate': results.detection_rate,
                    'path_length': results.path_length,
                    'efficiency': results.n_found / results.path_length
                })
            
            # Test zigzag paths
            for spacing in spacings:
                for angle in angles:
                    path = sim.zigzag_path(spacing=spacing, angle_degrees=angle)
                    results = sim.evaluate_path(path)
                    
                    self.results.append({
                        'trial': trial,
                        'pattern': 'zigzag',
                        'spacing': spacing,
                        'angle': angle,
                        'n_found': results.n_found,
                        'n_total': n_total_plants,
                        'detection_rate': results.detection_rate,
                        'path_length': results.path_length,
                        'efficiency': results.n_found / results.path_length
                    })
    
    def analyze_results(self):
        """Analyze and summarize parameter sweep results."""
        df = pd.DataFrame(self.results)
        
        # Analyze parallel patterns
        parallel_summary = df[df['pattern'] == 'parallel'].groupby('spacing').agg({
            'detection_rate': ['mean', 'std'],
            'path_length': ['mean', 'std'],
            'efficiency': ['mean', 'std']
        })
        
        # Analyze zigzag patterns
        zigzag_summary = df[df['pattern'] == 'zigzag'].groupby(['spacing', 'angle']).agg({
            'detection_rate': ['mean', 'std'],
            'path_length': ['mean', 'std'],
            'efficiency': ['mean', 'std']
        })
        
        # Find best parameters
        best_parallel = parallel_summary['detection_rate']['mean'].idxmax()
        best_zigzag = zigzag_summary['detection_rate']['mean'].idxmax()
        
        return {
            'parallel_summary': parallel_summary,
            'zigzag_summary': zigzag_summary,
            'best_parallel_spacing': best_parallel,
            'best_zigzag_params': best_zigzag,
            'full_results': df
        }
    
    def find_spacing_for_detection_rate(self, target_rate: float = 0.95, 
                                      pattern: str = 'parallel', 
                                      angle: float = None) -> dict:
        """
        Find the largest spacing that achieves a target detection rate.
        
        Args:
            target_rate: Target detection rate (0-1)
            pattern: Search pattern ('parallel' or 'zigzag')
            angle: Specific angle for zigzag pattern (optional)
            
        Returns:
            Dictionary containing results of the analysis
        """
        df = pd.DataFrame(self.results)
        
        # Filter by pattern
        pattern_data = df[df['pattern'] == pattern].copy()
        
        # For zigzag, handle angle selection
        if pattern == 'zigzag':
            if angle is not None:
                pattern_data = pattern_data[pattern_data['angle'] == angle]
            else:
                # For each spacing, use the angle that gives the best rate
                pattern_data = pattern_data.groupby(['spacing', 'angle'])['detection_rate'].mean().reset_index()
                best_angles = pattern_data.groupby('spacing')['detection_rate'].idxmax()
                pattern_data = pattern_data.loc[best_angles]
        
        # Calculate mean detection rate for each spacing
        spacing_results = pattern_data.groupby('spacing')['detection_rate'].mean()
        
        # Sort spacings by size (descending)
        sorted_spacings = sorted(spacing_results.index, reverse=True)
        
        # Find the largest spacing that meets or exceeds the target rate
        for spacing in sorted_spacings:
            rate = spacing_results[spacing]
            if rate >= target_rate:
                if pattern == 'zigzag' and angle is None:
                    # Find the best angle for this spacing
                    angle_data = df[
                        (df['pattern'] == 'zigzag') & 
                        (df['spacing'] == spacing)
                    ]
                    best_angle = angle_data.groupby('angle')['detection_rate'].mean().idxmax()
                    actual_rate = angle_data[angle_data['angle'] == best_angle]['detection_rate'].mean()
                    
                    return {
                        'success': True,
                        'spacing': spacing,
                        'angle': best_angle,
                        'detection_rate': actual_rate,
                        'message': f'Largest spacing achieving {target_rate:.1%} detection rate'
                    }
                else:
                    return {
                        'success': True,
                        'spacing': spacing,
                        'detection_rate': rate,
                        'message': f'Largest spacing achieving {target_rate:.1%} detection rate'
                    }
        
        return {
            'success': False,
            'message': f'No spacing achieves {target_rate:.1%} detection rate',
            'max_achievable_rate': spacing_results.max(),
            'spacing': None,
            'detection_rate': None
        }
    
    def plot_marginal_analysis(self):
        """Plot marginal returns analysis."""
        df = pd.DataFrame(self.results)
        
        # Create figure with two subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Analyze parallel pattern
        parallel_data = df[df['pattern'] == 'parallel']
        mean_rates = parallel_data.groupby('spacing')['detection_rate'].mean()
        spacings = sorted(mean_rates.index)
        rates = [mean_rates[s] for s in spacings]
        
        ax1.plot(spacings, rates, 'bo-')
        ax1.set_title('Parallel Pattern: Detection Rate vs Spacing')
        ax1.set_xlabel('Spacing (m)')
        ax1.set_ylabel('Detection Rate')
        ax1.grid(True)
        
        # Analyze zigzag pattern for each angle
        zigzag_data = df[df['pattern'] == 'zigzag']
        for angle in sorted(zigzag_data['angle'].unique()):
            angle_data = zigzag_data[zigzag_data['angle'] == angle]
            mean_rates = angle_data.groupby('spacing')['detection_rate'].mean()
            spacings = sorted(mean_rates.index)
            rates = [mean_rates[s] for s in spacings]
            
            ax2.plot(spacings, rates, 'o-', label=f'Angle: {angle}°')
            
        ax2.set_title('Zigzag Pattern: Detection Rate vs Spacing')
        ax2.set_xlabel('Spacing (m)')
        ax2.set_ylabel('Detection Rate')
        ax2.grid(True)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
    
    def plot_best_patterns(self):
        """Plot the best-performing patterns."""
        results = self.analyze_results()
        
        # Plot parallel pattern
        plt.figure(figsize=(10, 10))
        sim = PlotSimulation(**self.plot_params)
        best_parallel = sim.parallel_path(spacing=results['best_parallel_spacing'])  # Create parallel path
        parallel_results = sim.evaluate_path(best_parallel)
        print(f"\nBest Parallel Pattern (spacing={results['best_parallel_spacing']:.1f}m):")
        print(f"Detection Rate: {parallel_results.detection_rate:.1%}")
        print(f"Path Length: {parallel_results.path_length:.1f}m")
        sim.plot_simulation(best_parallel, parallel_results.found_plants,
                          title=f"Best Parallel Pattern (spacing={results['best_parallel_spacing']:.1f}m)")
        
        # Create new plot for zigzag pattern
        plt.figure(figsize=(10, 10))
        sim = PlotSimulation(**self.plot_params)  # Fresh simulation
        best_spacing, best_angle = results['best_zigzag_params']
        best_zigzag = sim.zigzag_path(spacing=best_spacing, angle_degrees=best_angle)  # Create zigzag path
        zigzag_results = sim.evaluate_path(best_zigzag)
        print(f"\nBest Zigzag Pattern (spacing={best_spacing:.1f}m, angle={best_angle}°):")
        print(f"Detection Rate: {zigzag_results.detection_rate:.1%}")
        print(f"Path Length: {zigzag_results.path_length:.1f}m")
        sim.plot_simulation(best_zigzag, zigzag_results.found_plants,
                          title=f"Best Zigzag Pattern (spacing={best_spacing:.1f}m, angle={best_angle}°)")
        
        return results
    
    @staticmethod
    def run_distribution_comparison(n_trials: int = 5,
                                    detection_params: float = 0.2,
                                    n_clusters_params: int = 3,
                                    cluster_std_params: float = 1.0,
                                    jitter_params: float = 0.3,
                                    steepness_params: float = 2.0,
                                    spacings_params: List[float] = [0.5, 1.0, 1.5, 2.0, 2.5],
                                    angles_params: List[float] = [0, 15, 30, 45]):
        """
        Compare different distribution types with various search patterns.
        
        Args:
            n_trials: Number of trials per configuration
            spacings: List of spacing values to test
            angles: List of angles to test for zigzag patterns
        """
        # Define distributions to test
        distributions = {
            'random': {
                'distribution': 'random',
                'detection_params': {'fixed': detection_params}
            },
            'clustered': {
                'distribution': 'clustered',
                'distribution_params': {'n_clusters': n_clusters_params, 'cluster_std': cluster_std_params},
                'detection_params': {'fixed': detection_params}
            },
            'regular': {
                'distribution': 'regular',
                'distribution_params': {'jitter': jitter_params},
                'detection_params': {'fixed': detection_params}
            },
            'gradient': {
                'distribution': 'gradient',
                'distribution_params': {'direction': 'x', 'steepness': steepness_params},
                'detection_params': {'fixed': detection_params}
            }
        }
        
        # Store results for each distribution
        all_results = {}
        
        for dist_name, dist_params in distributions.items():
            print(f"\nTesting {dist_name} distribution...")
            optimizer = SearchOptimization({
                'n_plants': 30,
                **dist_params
            })
            
            optimizer.run_parameter_sweep(n_trials, spacings_params, angles_params)
            all_results[dist_name] = optimizer.results
        
        return all_results
    
    @staticmethod
    def plot_distribution_comparison(all_results):
        """Plot comparison of detection rates across different distributions."""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Colors for different distributions
        colors = {'random': 'blue', 'clustered': 'red', 
                 'regular': 'green', 'gradient': 'purple'}
        
        # Plot parallel patterns
        for dist_name, results in all_results.items():
            df = pd.DataFrame(results)
            parallel_data = df[df['pattern'] == 'parallel']
            
            mean_rates = parallel_data.groupby('spacing')['detection_rate'].mean()
            spacings = sorted(mean_rates.index)
            rates = [mean_rates[s] for s in spacings]
            
            ax1.plot(spacings, rates, 'o-', 
                    label=f'{dist_name.capitalize()}',
                    color=colors[dist_name])
        
        ax1.set_title('Parallel Pattern: Detection Rate by Distribution')
        ax1.set_xlabel('Spacing (m)')
        ax1.set_ylabel('Detection Rate')
        ax1.grid(True)
        ax1.legend()
        
        # Plot zigzag patterns (using best angle for each distribution)
        for dist_name, results in all_results.items():
            df = pd.DataFrame(results)
            zigzag_data = df[df['pattern'] == 'zigzag']
            
            angle_performance = zigzag_data.groupby('angle')['detection_rate'].mean()
            best_angle = angle_performance.idxmax()
            
            best_angle_data = zigzag_data[zigzag_data['angle'] == best_angle]
            mean_rates = best_angle_data.groupby('spacing')['detection_rate'].mean()
            spacings = sorted(mean_rates.index)
            rates = [mean_rates[s] for s in spacings]
            
            ax2.plot(spacings, rates, 'o-',
                    label=f'{dist_name.capitalize()} (angle={best_angle}°)',
                    color=colors[dist_name])
        
        ax2.set_title('Zigzag Pattern: Detection Rate by Distribution')
        ax2.set_xlabel('Spacing (m)')
        ax2.set_ylabel('Detection Rate')
        ax2.grid(True)
        ax2.legend()

        plt.tight_layout()
        plt.show()

In [ ]:
# Run distribution comparison
print("Running distribution comparison...")
results = SearchOptimization.run_distribution_comparison(
    n_trials=100,
    detection_params=2.0,
    spacings_params=[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0],
    angles_params=[0, 15, 30, 45]
)

SearchOptimization.plot_distribution_comparison(results)

In [ ]:
# Create and run optimization
optimizer = SearchOptimization({
    'n_plants': 200,
    'distribution': 'random',
    'detection_params': {'fixed': 2.0}
})

# Run parameter sweep
optimizer.run_parameter_sweep(spacings=[1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0])

# Plot best patterns
results = optimizer.plot_best_patterns()

# Find spacing for different detection rates
target_rate = 0.90  # 90% detection rate
result = optimizer.find_spacing_for_detection_rate(target_rate=target_rate, pattern='parallel')
print(f"\nMaximum spacing for {target_rate:.1%} detection rate:")
print(f"Spacing: {result['spacing']:.2f}m")
print(f"Actual detection rate: {result['detection_rate']:.2%}")

print(results)